#Libraries

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from google.colab import drive

##Fot the tree

In [ ]:
from sklearn.tree import export_graphviz
import pydot

##For collab

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


#Open the dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Agrimonia/Parte2/Recap previous thesis/Agrimonia_Dataset_v_2_0_1.csv")
pd.set_option('display.max_columns', None)
df['Latitude'] = df['Latitude'].round(2)
df['Longitude'] = df['Longitude'].round(2)

df.head()

In [ ]:
print(df.dtypes)

In [ ]:
#df['IDStations'] = df['IDStations'].astype(float)
df['AQ_pm10'] = df['AQ_pm10'].astype(float)
df['AQ_pm25'] = df['AQ_pm25'].astype(float)
df['AQ_co'] = df['AQ_co'].astype(float)
df['AQ_nh3'] = df['AQ_nh3'].astype(float)
df['AQ_nox'] = df['AQ_nox'].astype(float)
df['AQ_no2'] = df['AQ_no2'].astype(float)
df['AQ_so2'] = df['AQ_so2'].astype(float)

df['EM_nh3_livestock_mm'] = df['EM_nh3_livestock_mm'].astype(float)
df['EM_nh3_agr_soils'] = df['EM_nh3_agr_soils'].astype(float)
df['EM_nh3_agr_waste_burn'] = df['EM_nh3_agr_waste_burn'].astype(float)
df['EM_nh3_sum'] = df['EM_nh3_sum'].astype(float)
df['EM_nox_traffic'] = df['EM_nox_traffic'].astype(float)
df['EM_nox_sum'] = df['EM_nox_sum'].astype(float)
df['EM_so2_sum'] = df['EM_so2_sum'].astype(float)
df['LI_pigs'] = df['LI_pigs'].astype(float)
df['LI_bovine'] = df['LI_bovine'].astype(float)

In [ ]:
print(df.dtypes)

#Select one station

In [ ]:
mydf = df[df['AQ_nh3'].notna()& df['AQ_pm10'].notna()]
mydf = mydf[mydf['IDStations']==1266]
mydf = mydf.drop(['IDStations','AQ_pm25', 'AQ_co', 'AQ_so2', 'Time', 'WE_mode_wind_direction_10m', 'WE_mode_wind_direction_100m'], axis=1)
print(mydf.shape)
mydf

In [ ]:
print(mydf.isna().sum())

## Remove NaN

In [ ]:
mydf.dropna(inplace=True)
mydf

In [ ]:
print(mydf.isna().sum())

#Random forest model

In [ ]:
y = mydf['AQ_pm10']
print(type(y))
print(y)

In [ ]:
X = mydf.drop(['AQ_pm10'], axis=1)
X = X.dropna()

y = mydf['AQ_pm10']
y = y.dropna()

In [ ]:
# Divisione dei dati in set di addestramento e set di test
X = mydf.drop(['AQ_pm10'], axis=1)
X = X.dropna()

y = mydf['AQ_pm10']
y = y.dropna()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Creazione del modello Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Addestramento del modello
rf_model.fit(X_train, y_train)

# Valutazione delle prestazioni del modello
train_score = rf_model.score(X_train, y_train)
test_score = rf_model.score(X_test, y_test)
print('R-squared sul set di addestramento:', train_score)
print('R-squared sul set di test:', test_score)

y_pred_train = rf_model.predict(X_train)
y_pred_test = rf_model.predict(X_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
print('MAE sul set di addestramento:', train_mae)
print('MAE sul set di test:', test_mae)

# Variabili più significative
importances = rf_model.feature_importances_
variables = X.columns
important_variables = pd.DataFrame({'Variabile': variables, 'Importanza': importances})
important_variables = important_variables.sort_values('Importanza', ascending=False)
print(important_variables)


# Feature importance based on feature permutation

In [ ]:
# Feature importance based on feature permutation
importance = permutation_importance(rf_model, X_test, y_test, n_repeats=10, random_state=42)

# Sorting the feature importance in descending order
indices = np.argsort(importance.importances_mean)[::-1]
important_variables = pd.DataFrame({'Variabile': X.columns[indices], 'Importanza': importance.importances_mean[indices]})
print(important_variables)


#Print the tree

In [ ]:
from sklearn.tree import export_text

tree = rf_model.estimators_[0]
tree_text = export_text(tree, feature_names=X.columns.tolist())
print(tree_text)

In [ ]:
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import display, Image

tree = rf_model.estimators_[0]

dot_data = export_graphviz(tree, out_file=None, 
                           feature_names=X.columns,  
                           filled=True, rounded=True,  
                           special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data)
display(Image(graph.create_png()))

In [ ]:
dot_data = export_graphviz(tree, out_file=None, 
                           feature_names=X.columns,  
                           filled=True, rounded=True,  
                           special_characters=True,
                           max_depth=3)

In [ ]:

graph = pydotplus.graph_from_dot_data(dot_data)
display(Image(graph.create_png()))